### 7.1.2 使用 FFT 获得功率频谱密度估计

此示例说明如何使用 periodogram 和 fft 函数获得非参数化功率谱密度 (PSD) 估计。这些不同用例说明针对偶数长度输入、归一化频率和以赫兹表示的频率以及单边和双边 PSD 估计，如何正确缩放 fft 的输出。所有用例都使用矩形窗。
#### 具有指定采样率的偶数长度输入
对于一个采样率为1 kHz的偶数长度信号，分别使用fft和periodogram获得其周期图。比较二者的结果。

创建一个含 N(0,1) 加性噪声的100 Hz正弦波信号。采样频率为 1 kHz。信号长度为1000个采样。

使用 fft 获取周期图。信号是偶数长度的实数值信号。由于信号是实数值信号，您只需要对正负频率之一进行功率估计。为了保持总功率不变，将同时在两组（正频率和负频率）中出现的所有频率乘以因子 2。零频率 (DC) 和奈奎斯特频率不会出现两次。绘制结果。

In [2]:
using TySignalProcessing
using TySystemIdentification
using TyPlot
using TyMath
fs = 1000
t = 0:1/fs:1-1/fs
x = cos.(2*pi*100*t) + randn(size(t))
N = length(x)
xdft = fft(x)
xdft = xdft[1:Int(N/2)+1]
psdx = (1/(fs*N)) * abs.(xdft).^2
psdx[2:end-1] = 2*psdx[2:end-1]
freq = 0:fs/length(x):fs/2
plot(freq,pow2db(psdx))
grid("on")
title("Periodogram Using FFT")
xlabel("Frequency (Hz)")
ylabel("Power/Frequency (dB/Hz)")



PyObject <objects.mw_text.CYlabel object at 0x000001DAF66BD088>

计算并使用 periodogram 绘制周期图。二者的结果相同。

In [3]:
figure()
periodogram(x,rectwin(N),N,fs;plotfig=true)

In [4]:
mxerr = max(psdx'-periodogram(x,rectwin(N),N,fs))

DimensionMismatch: DimensionMismatch: dimensions must match: a has dims (Base.OneTo(1), Base.OneTo(501)), b has dims (Base.OneTo(501), Base.OneTo(1)), mismatch at 1

#### 具有归一化频率的输入
通过 fft 为使用归一化频率的输入生成周期图。创建一个带 N(0,1) 加性噪声的正弦波信号。该正弦波的角频率为 π/4 弧度/采样点。


In [5]:
N = 1000
n = 0:N-1
x = cos.(pi/4*n) + randn(size(n))

1000-element Vector{Float64}:
  1.3410290451725666
  2.4966992666408285
 -0.3791268867848153
 -0.9448728205218557
 -1.108884519866134
 -0.7142494030689829
 -0.05485633170454848
  1.6752563494012715
  1.3938938042848568
  0.42194183066137414
  ⋮
  1.3861170141927475
 -0.1454308520136367
  2.433895795690438
 -0.6036965946369771
 -0.13657919031507704
 -1.2686657871196658
 -0.6432330831931448
  0.02490386236063374
 -1.9526466018786843

使用 fft 获取周期图。信号是偶数长度的实数值信号。由于信号是实数值信号，您只需要对正负频率之一进行功率估计。为了保持总功率不变，将同时在两组（正频率和负频率）中出现的所有频率乘以因子 2。零频率 (DC) 和奈奎斯特频率不会出现两次。绘制结果。

In [6]:
xdft = fft(x)
xdft = xdft[1:Int(N/2+1)]
psdx = (1/(2*pi*N)) * abs.(xdft).^2
psdx[2:end-1] = 2*psdx[2:end-1]
freq = 0:2*pi/N:pi
plot(freq/pi,pow2db(psdx))
grid("on")
title("Periodogram Using FFT")
xlabel("Normalized Frequency (*pi rad/sample)")
ylabel("Power/Frequency (dB/(rad/sample))")

PyObject <objects.mw_text.CYlabel object at 0x000001DA8B0FCA88>

计算并使用 periodogram 绘制周期图。二者的结果相同。

In [7]:
figure()
periodogram(x,rectwin(N),N;plotfig=true)

In [8]:
mxerr = maximum(psdx-periodogram(x,rectwin(N),N))

4.440892098500626e-16

#### 具有归一化频率的复数值输入
使用 fft 为具有归一化频率的复数值输入生成周期图。采用一个带 N(0,1) 复噪声的复指数信号，角频率为 π/4 弧度/采样点。

In [9]:
N = 1000
n = 0:N-1
x = exp.(im*pi/4*n) + vec([1 im]*randn(2,N)/sqrt(2))

1000-element Vector{ComplexF64}:
   1.0607678814790622 - 0.35233240411264755im
   1.4435955088940085 + 1.6623651365493397im
   0.9753929832240522 + 2.2612443326241296im
  -0.1479472183413585 + 0.6321513028847715im
  0.09890499230653504 + 0.5703549010879081im
 -0.20395971830818027 - 0.3603121745528047im
   0.8820470018584111 - 1.5477867347491474im
   1.8977040399583731 - 1.2605958105392956im
   0.6966772470333978 - 0.464499281376724im
    0.153178852536215 + 1.4053775136394886im
                      ⋮
   1.4117911715917284 - 0.9885662596148965im
    1.119279558491381 + 1.3962317188581672im
    1.501137926273075 + 0.707875506021572im
  -0.8854084303669956 + 1.090803732821077im
  -0.0841106568851282 + 0.0978243883340092im
  -1.7516870024139597 - 0.3977643224878448im
  -0.5435855296360504 - 0.9719826442877206im
  0.12585763427053887 - 1.3001406488711804im
   0.6245266666215199 - 0.35425577415249837im

使用 fft 获得周期图。由于输入是复数值，此处求 [0,2π) 弧度/采样点区间内的周期图。绘制结果。

In [10]:
xdft = fft(x)
psdx = (1/(2*pi*N)) * abs.(xdft).^2
freq = 0:2*pi/N:2*pi-2*pi/N
plot(freq/pi,pow2db(psdx))
grid("on")
title("Periodogram Using FFT")
xlabel("Normalized Frequency (*pi rad/sample)")
ylabel("Power/Frequency (dB/(rad/sample))")

PyObject <objects.mw_text.CYlabel object at 0x000001DA8A0E0EC8>

计算并使用 periodogram 绘制周期图。二者的结果相同。

In [11]:
figure()
periodogram(x,rectwin(N),N,"twosided";plotfig=true)

In [12]:
mxerr = maximum(psdx-periodogram(x,rectwin(N),N,"twosided"))

2.220446049250313e-16